Вариант 1

Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность) с сайтов HH(обязательно) и/или Superjob(по желанию). Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

Наименование вакансии.

Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).

Ссылку на саму вакансию.

Сайт, откуда собрана вакансия.

По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas. Сохраните в json либо csv.

In [18]:
from bs4 import BeautifulSoup as bs
import requests
import re
from time import sleep

In [19]:
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'

URL_FIRST_PAGE_HH = 'https://hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=аналитик&no_magic=true&L_save_area=true&items_on_page=20'


In [20]:
headers = {
    'User-Agent': USER_AGENT,
}

In [21]:
def parse_hh(url_page, headers, result=[], index_page=1):
    response = requests.get(url_page, headers=headers)
    if response.status_code != 200:
        print('Парсинг завершен')
        return result
    else:
        print('Cтраница №%d, ссылка: %s'%(index_page, response.url))

    dom = bs(response.content, 'html.parser')
    vacancies = dom.find_all('div', {'class': 'vacancy-serp-item__layout'})
    for vacancy in vacancies:
        result.append(parse_vacancy_hh(vacancy))

    link_next_page = dom.find('a', {'data-qa': 'pager-next'})
    if link_next_page:
        link_next_page =  'https://hh.ru' + link_next_page['href']
    else:
        print('Парсинг завершен')
        return result

    result = parse_hh(link_next_page, headers, result, index_page+1)   
    return result

def parse_vacancy_hh(dom_vacancy):
    sleep(0.1)

    vacancy_name = dom_vacancy.find('a', {'class': 'serp-item__title'}).getText()
    vacancy_link = dom_vacancy.find('a', {'class': 'serp-item__title'})['href']
    vacancy_salary = dom_vacancy.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
    if vacancy_salary is not None:
        if "до" in vacancy_salary.getText():
            vacancy_salary = "Максимальная зарплата = " + vacancy_salary.getText().replace('\u202f', ' ')
        else:
            if "от" in vacancy_salary.getText():
                vacancy_salary = "Минимальная зарплата = " + vacancy_salary.getText().replace('\u202f', ' ')
            else:
                vacancy_salary = "Средняя зарплата = " + vacancy_salary.getText().replace('\u202f', ' ')
    else:
        vacancy_salary = 'Зарплата не указана' 
                
    return {
      'name': vacancy_name,
      'link': vacancy_link,
      'salary': vacancy_salary,
        }    

In [22]:
result = parse_hh(URL_FIRST_PAGE_HH, headers)

Cтраница №1, ссылка: https://hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA&no_magic=true&L_save_area=true&items_on_page=20
Cтраница №2, ссылка: https://hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA&no_magic=true&L_save_area=true&items_on_page=20&page=1&hhtmFrom=vacancy_search_list
Cтраница №3, ссылка: https://hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA&no_magic=true&L_save_area=true&items_on_page=20&page=2&hhtmFrom=vacancy_search_list
Cтраница №4, ссылка: https://hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA&no_magic=true&L_save_area=true&items_on_page=20&page=3&hhtmF

In [23]:
len(result)

926

In [24]:
result

[{'name': 'Аналитик',
  'link': 'https://hh.ru/vacancy/76292601?from=vacancy_search_list&query=%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA',
  'salary': 'Зарплата не указана'},
 {'name': 'Аналитик',
  'link': 'https://hh.ru/vacancy/77187761?from=vacancy_search_list&query=%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA',
  'salary': 'Минимальная зарплата = от 60 000 руб.'},
 {'name': 'Аналитик',
  'link': 'https://hh.ru/vacancy/75610210?from=vacancy_search_list&query=%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA',
  'salary': 'Средняя зарплата = 52 000 – 60 000 руб.'},
 {'name': 'Аналитик (по работе с облачными базами данных)',
  'link': 'https://hh.ru/vacancy/74251838?from=vacancy_search_list&query=%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA',
  'salary': 'Средняя зарплата = 50 000 – 100 000 руб.'},
 {'name': 'Аналитик бизнес процессов',
  'link': 'https://hh.ru/vacancy/76279241?from=vacancy_search_list&query=%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA',
  'salary': 